# 0) Install and import all the needed dependencies

You need to start by installing three packages: mediapipe, opencv and scikit.

NOTE: Once you have run this first block once you will not need to run it again.
This just installs these packages onto your local machine.


In [ ]:
!pip install mediapipe==0.10.9 opencv-python scikit-learn

# 0)  Make some detections 

Run this if you want to make sure you have dependencies installed correctly.

<!-- Hand Landmark Image Map <img src="https://i.imgur.com/8bForKY.png">-->
POSE GESTURE LANDMARKS GUIDE
<img src="https://i.imgur.com/AzKNp7A.png">

In [ ]:
import mediapipe as mp #Imports mediapipe
import cv2 #Imports open cv
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #precents copying data but still allows use of image for rendering
        
        # Make Detections
        results = holistic.process(image)
                
        # Recolor image back to BGR for rendering
        image.flags.writeable = True #precents copying data but still allows use of image for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Right hand - Draw the right hand keypoints
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Left Hand - Draw the left hand keypoints
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections - Draw the body keypoints as pictured below
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 1) CAPTURE LANDMARK DATA FOR A POSE AND SAVE IT TO A SPREADSHEET

This creates a spreadsheet with a number of columns equal to the number of keypoints in your pose (33 total - it ignores the hands and face in the holistic model).

In [ ]:
#Button 1 - CREATE SPREADSHEET
import csv
import os
import numpy as np
num_coords = len(results.pose_landmarks.landmark)
num_coords
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]
with open('data.csv', mode='w',newline='') as f:
    csv_writer = csv.writer(f, delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
#Button 2 - SET POSE NAME
class_name = input("Enter the name for the pose : ")

In [ ]:
#Button 3 - COLLECT POSE DATA
import csv
import os
import numpy as np
num_coords = len(results.pose_landmarks.landmark)
num_coords
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]
with open('data.csv', mode='w',newline='') as f:
    csv_writer = csv.writer(f, delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)
    
class_name = input("Enter the name for the pose : ")

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False #prevents copying data but still allows use of image for rendering
        
        # Make Detections
        results = holistic.process(image)        
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw the Right hand - Purely for show. The model doesn't use these in fitting an ML model
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Draw the Left Hand - Purely for show. The model doesn't use these in fitting an ML model
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections - This is where the magic is! These are the points you'll use
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        # Export coordinates
        try:
            #Extract pose and face landmarks and put into numpy arrays
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            #print(pose_row) #Uncomment to output the poserow data to the screen

            #concatenate rows
            row = pose_row
            
            #append the class name to the row in the zeroth column
            row.insert(0, class_name)
            
            #export to csv
            with open('data.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            print("NERP MAGERP")
            pass
        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## 2) READ SPREADSHEET INTO A DATAFRAME

Takes the spreadsheet from section 1 and reads it into a dataframe

In [3]:
#Button 4 - IMPORT DATA
import pandas as pd
from sklearn.model_selection import train_test_split # allows you to make a training partition and a testing partition with your data
df = pd.read_csv('acsm.csv') # reads in your coords.csv file into a dataframe
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
6306,Right Tandem,0.400500,0.247072,0.706913,0.997826,0.391777,0.246111,0.754640,0.994349,0.386644,...,-1.394353,0.440951,0.388914,0.604618,-1.607975,0.399085,0.658242,0.610542,-1.629802,0.411492
6307,Right Tandem,0.416785,0.202730,-1.035553,0.997976,0.421214,0.159012,-1.065934,0.994844,0.408643,...,0.642470,0.422563,0.565718,0.604398,0.502094,0.418193,0.580037,0.604433,0.480790,0.418706
6308,Right Tandem,0.701842,0.504487,-0.151704,0.997313,0.697802,0.530398,-0.169159,0.994737,0.694386,...,0.377917,0.391942,0.556934,0.011496,0.278288,0.385345,0.516009,-0.020136,0.381343,0.381730
6309,Right Tandem,0.466610,0.378551,0.214004,0.987408,0.476638,0.377616,0.202578,0.980648,0.478699,...,0.069763,0.426605,0.625711,0.377789,0.197643,0.363624,0.601898,0.339870,0.083818,0.377703
6310,Right Tandem,0.562592,0.587223,0.641245,0.982282,0.566940,0.593824,0.636792,0.977000,0.569268,...,-0.282563,0.478619,0.535416,0.075285,-0.200769,0.395817,0.551226,0.039313,-0.273036,0.414886


## PREPROCESS DATA

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
import joblib
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

def augment_data_advanced(df, feature_names, augmentation_factor=0.1):
    augmented_data = []
    for index, row in df.iterrows():
        augmented_row = row.copy()
        for feature in feature_names:
            if np.random.rand() < 0.5:
                augmented_row[feature] *= (1 + np.random.uniform(-augmentation_factor, augmentation_factor))
        augmented_data.append(augmented_row)
    return pd.DataFrame(augmented_data)

# Load your dataframe
# df = pd.read_csv('your_data.csv')  # Uncomment if reading from a CSV

# Define feature names for keypoints 25 to 32
selected_keypoints = [f'{axis}{i}' for i in range(25, 33) for axis in ['x', 'y', 'z', 'v']]

# Check for missing values and drop rows with any missing values
df = df.dropna()

# Normalize the keypoint data
scaler = StandardScaler()
df[selected_keypoints] = scaler.fit_transform(df[selected_keypoints])

# Save the scaler for use on live data
joblib.dump(scaler, 'scaler.joblib')

# Calculate z-scores for the features with a more conservative threshold
z_scores = np.abs(stats.zscore(df[selected_keypoints]))
threshold = 3.5  # More conservative threshold

# Filter out the rows with any z-score greater than the threshold
df = df[(z_scores < threshold).all(axis=1)]

# Ensure all classes are represented
class_counts = df['class'].value_counts()
min_count = class_counts.min()
df_balanced = pd.concat([resample(df[df['class'] == cls], replace=True, n_samples=min_count, random_state=123) for cls in class_counts.index])

# Augment the data
df_augmented = augment_data_advanced(df_balanced, selected_keypoints)
df = pd.concat([df_balanced, df_augmented], ignore_index=True)

# Dimensionality reduction with PCA
pca = PCA(n_components=0.95)
principal_components = pca.fit_transform(df[selected_keypoints])

# Save PCA for later use
joblib.dump(pca, 'pca.joblib')

df_pca = pd.DataFrame(data=principal_components)
df_pca['class'] = df['class'].values

# Feature selection
clf = RandomForestClassifier(n_estimators=100, random_state=123)
clf.fit(df_pca.iloc[:, :-1], df_pca['class'])

selector = SelectFromModel(clf, prefit=True)
selected_features_mask = selector.get_support()

# Since PCA transforms the original features, we need to identify the selected PCA components
pca_components_selected = np.where(selected_features_mask)[0]

# Print the indices of the selected PCA components
print(f"Selected {pca_components_selected.shape[0]} PCA components out of {df_pca.shape[1] - 1}")
print("Selected PCA components indices:", pca_components_selected)

# Transform the PCA data to include only the selected features
df_selected = pd.DataFrame(data=selector.transform(df_pca.iloc[:, :-1]))
df_selected['class'] = df_pca['class'].values

# Now your data is preprocessed and ready for modeling
print("Data preprocessing complete. Here's a preview of the processed data:")
print(df_selected.tail())

Selected 3 PCA components out of 6
Selected PCA components indices: [0 1 2]
Data preprocessing complete. Here's a preview of the processed data:
             0         1         2        class
9745 -4.044725  1.621444  1.900983  Left Single
9746  2.725031  3.366933  2.321481  Left Single
9747 -3.740199  1.123429  1.769508  Left Single
9748  2.571298  3.248768  2.752696  Left Single
9749 -3.504421  0.051509  1.052343  Left Single


In [5]:
selected_keypoints

['x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32']

## 3) CREATE AN ML MODEL FOR LIVE CLASSIFICATION

This will use the dataframe created in section 3 to generate multiple machine learning models (Logistic Regression, Ridge Regression, Random Forest and Gradient Descent). It will analyze these models and choose the best performing one and output into a file using JOBLIB for later use on live classification.

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import joblib
from scipy.stats import randint

# Define feature names for keypoints 25 to 32
selected_keypoints = [f'{axis}{i}' for i in range(25, 33) for axis in ['x', 'y', 'z', 'v']]
X = df_selected.iloc[:, :-1]
y = df_selected['class']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234, stratify=y)

# Cross-validation with default RandomForest
rf_model = RandomForestClassifier(random_state=123)
rf_cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
print(f'RandomForest CV Scores: {rf_cv_scores.mean():.4f}')

# Fit the model with the training data
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Evaluate the default RandomForest model
print('Default RandomForest Classification Report:')
print(classification_report(y_test, rf_pred))

# Hyperparameter tuning with RandomizedSearchCV
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11),
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(RandomForestClassifier(random_state=123), param_distributions=param_dist, n_iter=50, cv=5, random_state=123, n_jobs=-1, verbose=2)
random_search.fit(X_train, y_train)

# Get the best model from random search
best_rf_model = random_search.best_estimator_
rf_tuned_pred = best_rf_model.predict(X_test)

print(f'Best parameters found for RandomForest: {random_search.best_params_}')
print(f'Best cross-validation score for RandomForest: {random_search.best_score_:.4f}')

# Evaluate the tuned RandomForest model
print('Tuned RandomForest Classification Report:')
print(classification_report(y_test, rf_tuned_pred))

# Save the best performing model
joblib.dump(best_rf_model, 'best_tuned_rf_model.joblib')
print("Best tuned RandomForest model saved as 'best_tuned_rf_model.joblib'")

RandomForest CV Scores: 0.9963
Default RandomForest Classification Report:
              precision    recall  f1-score   support

  Double Leg       1.00      1.00      1.00       585
 Left Single       1.00      1.00      1.00       585
 Left Tandem       0.99      1.00      1.00       585
Right Single       1.00      1.00      1.00       585
Right Tandem       1.00      1.00      1.00       585

    accuracy                           1.00      2925
   macro avg       1.00      1.00      1.00      2925
weighted avg       1.00      1.00      1.00      2925

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found for RandomForest: {'bootstrap': False, 'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 154}
Best cross-validation score for RandomForest: 0.9953
Tuned RandomForest Classification Report:
              precision    recall  f1-score   support

  Double Leg       1.00      1.00      1.00       585
 Left Single       1.00 

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Load your dataframe
df = pd.read_csv('coords.csv')  # Replace with your actual file

# Define feature names for keypoints 25 to 32
selected_keypoints = [f'{axis}{i}' for i in range(25, 33) for axis in ['x', 'y', 'z', 'v']]
selected_features = selected_keypoints + ['class']

# Limit the DataFrame to the selected features
df = df[selected_features]

# Split the data
X = df[selected_keypoints]
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234, stratify=y)

# Train the RandomForest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=123)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print('Classification Report:')
print(classification_report(y_test, y_pred))

print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

# Save the trained model
joblib.dump(rf_model, 'simplified_rf_model.joblib')
print("Model saved as 'simplified_rf_model.joblib'")

Classification Report:
                         precision    recall  f1-score   support

      Double Leg Stance       1.00      1.00      1.00      1032
 Left Single Leg Stance       1.00      1.00      1.00       757
     Left Tandem Stance       1.00      1.00      1.00      1028
Right Single Leg Stance       1.00      1.00      1.00       879
    Right Tandem Stance       1.00      1.00      1.00      1037

               accuracy                           1.00      4733
              macro avg       1.00      1.00      1.00      4733
           weighted avg       1.00      1.00      1.00      4733

Confusion Matrix:
[[1032    0    0    0    0]
 [   0  756    0    1    0]
 [   0    0 1028    0    0]
 [   0    0    0  879    0]
 [   0    0    0    0 1037]]
Model saved as 'simplified_rf_model.joblib'


## 4) LIVE POSE CLASSIFICATION

This will load the joblib model created above and apply it to a live camera feed for live classification.

In [13]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import joblib
from tkinter import messagebox

try:
    # Load the saved model
    model = joblib.load("simplified_rf_model.joblib")

    class_names = model.classes_

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    mp_drawing = mp.solutions.drawing_utils

    # Define feature names for keypoints 25 to 32
    selected_keypoints = [f'{axis}{i}' for i in range(25, 33) for axis in ['x', 'y', 'z', 'v']]

    cap = cv2.VideoCapture(0)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            keypoints = []
            for i in range(25, 33):
                landmark = results.pose_landmarks.landmark[i]
                keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        else:
            keypoints = [0] * len(selected_keypoints)

        keypoints_df = pd.DataFrame([keypoints], columns=selected_keypoints)
        
        pose_probabilities = model.predict_proba(keypoints_df)[0]
        threshold = 0.5

        font_scale_detected = 1.5
        font_thickness_detected = 2
        font = cv2.FONT_HERSHEY_SIMPLEX
        x, y = 10, 50

        detected_pose = np.argmax(pose_probabilities)
        detected_prob = pose_probabilities[detected_pose]

        if detected_prob >= threshold:
            detected_text = f'Detected Pose: {class_names[detected_pose]} ({detected_prob:.2f})'
        else:
            detected_text = 'No pose detected'

        (text_width, text_height), baseline = cv2.getTextSize(detected_text, font, font_scale_detected, font_thickness_detected)
        rect_x1, rect_y1 = x, y - text_height - baseline
        rect_x2, rect_y2 = x + text_width, y + baseline
        cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), cv2.FILLED)
        cv2.putText(frame, detected_text, (x, y), font, font_scale_detected, (255, 255, 255), font_thickness_detected, cv2.LINE_AA)

        # Prepare text for each class
        font_scale_table = 1
        font_thickness_table = 2
        table_start_y = y + text_height + 20  # Start position for the table
        line_height = 40  # Height between each line

        for i, class_name in enumerate(class_names):
            prob = pose_probabilities[i]
            text = f'{class_name}: {prob:.0%}'

            # Draw background rectangle for text
            (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale_table, font_thickness_table)
            rect_x1, rect_y1 = x, table_start_y + i * line_height - text_height - baseline
            rect_x2, rect_y2 = x + text_width, table_start_y + i * line_height + baseline
            cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), cv2.FILLED)

            # Display text
            cv2.putText(frame, text, (x, table_start_y + i * line_height), font, font_scale_table, (255, 255, 255), font_thickness_table, cv2.LINE_AA)

        # Draw the pose landmarks on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Display the frame
        cv2.imshow('Pose Recognition', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
except Exception as e:
    messagebox.showerror("Error", f"An error occurred: {e}")

I0000 00:00:1717124300.569608       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro


[CV] END bootstrap=False, max_depth=20, min_samples_leaf=10, min_samples_split=2, n_estimators=161; total time=   0.6s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=5, min_samples_split=2, n_estimators=118; total time=   0.4s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=5, min_samples_split=10, n_estimators=185; total time=   0.7s
[CV] END bootstrap=True, max_depth=30, min_samples_leaf=6, min_samples_split=2, n_estimators=125; total time=   0.3s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=5, min_samples_split=8, n_estimators=159; total time=   0.6s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=9, min_samples_split=3, n_estimators=62; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=9, min_samples_split=3, n_estimators=62; total time=   0.2s
[CV] END bootstrap=False, max_depth=None, min_samples_leaf=9, min_samples_split=3, n_estimators=136; total time=   0.5s
[CV] END bootstrap=False, max_depth=30, min_samples_l

## GUI


In [1]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import csv
import os
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import joblib
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

class PoseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Pose Recognition GUI")

        # Button 1 - CREATE SPREADSHEET
        self.button1 = tk.Button(root, text="CREATE SPREADSHEET", command=self.create_spreadsheet)
        self.button1.pack(pady=10)

        # Button 2 - SET POSE NAME
        self.button2 = tk.Button(root, text="SET POSE NAME", command=self.set_pose_name)
        self.button2.pack(pady=10)

        # Button 3 - COLLECT POSE DATA
        self.button3 = tk.Button(root, text="COLLECT POSE DATA", command=self.collect_pose_data)
        self.button3.pack(pady=10)

        # Button 4 - IMPORT DATA
        self.button4 = tk.Button(root, text="IMPORT DATA", command=self.import_data)
        self.button4.pack(pady=10)

        # Button 5 - CREATE A MODEL!
        self.button5 = tk.Button(root, text="CREATE A MODEL!", command=self.create_model)
        self.button5.pack(pady=10)

        # Button 6 - LIVE CLASSIFY!
        self.button6 = tk.Button(root, text="LIVE CLASSIFY!", command=self.live_classify)
        self.button6.pack(pady=10)
        
        self.pose_name = ""
        self.df = None

    def create_spreadsheet(self):
        try:
            num_coords = 33  # Assuming 33 landmarks, replace with the correct number if needed
            landmarks = ['class']
            for val in range(1, num_coords + 1):
                landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
            with open('data.csv', mode='w', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(landmarks)
            messagebox.showinfo("Success", "Spreadsheet created successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def set_pose_name(self):
        self.pose_name = simpledialog.askstring("Input", "Enter the name for the pose:")
        if self.pose_name:
            messagebox.showinfo("Success", f"Pose name set to: {self.pose_name}")

    def collect_pose_data(self):
        if not self.pose_name:
            messagebox.showerror("Error", "Please set the pose name first!")
            return

        cap = cv2.VideoCapture(0)
        mp_holistic = mp.solutions.holistic
        mp_drawing = mp.solutions.drawing_utils

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

                try:
                    pose = results.pose_landmarks.landmark
                    pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                    row = pose_row
                    row.insert(0, self.pose_name)

                    with open('data.csv', mode='a', newline='') as f:
                        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                        csv_writer.writerow(row)
                except:
                    pass

                cv2.imshow('Raw Webcam Feed', image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
        cv2.destroyAllWindows()

    def import_data(self):
        try:
            self.df = pd.read_csv('coords.csv')
            messagebox.showinfo("Success", "Data imported successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def create_model(self):
        if self.df is None:
            messagebox.showerror("Error", "Please import the data first!")
            return

        try:
            print("Building ML model")
            feature_names = [f'keypoint_{i}_{axis}' for i in range(33) for axis in ['x', 'y', 'z', 'visibility']]
            x = self.df.drop('class', axis=1)
            x.columns = feature_names
            y = self.df['class']

            X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234, stratify=y)

            models = {
                'Logistic Regression': LogisticRegression(max_iter=10000),
                'Ridge Classifier': RidgeClassifier(),
                'Random Forest': RandomForestClassifier(random_state=1234),
                'Gradient Boosting': GradientBoostingClassifier(random_state=1234)
            }

            pipelines = {name: make_pipeline(StandardScaler(), model) for name, model in models.items()}
            cv_scores = {name: cross_val_score(pipeline, X_train, y_train, cv=5) for name, pipeline in pipelines.items()}
            best_model_name = max(cv_scores, key=lambda name: cv_scores[name].mean())
            best_model = pipelines[best_model_name]
            best_model.fit(X_train, y_train)
            y_pred = best_model.predict(X_test)

            print('Classification Report:')
            print(classification_report(y_test, y_pred))
            print('Confusion Matrix:')
            print(confusion_matrix(y_test, y_pred))

            joblib_file_model = "best_tuned_model.joblib"
            joblib.dump(best_model, joblib_file_model)
            messagebox.showinfo("Success", f"Model created and saved as {joblib_file_model}!")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def live_classify(self):
        try:
            best_tuned_model = joblib.load("best_tuned_model.joblib")

            scaler = best_tuned_model.named_steps['standardscaler']
            model = best_tuned_model.named_steps[best_tuned_model.steps[-1][0]]

            class_names = model.classes_

            mp_pose = mp.solutions.pose
            pose = mp_pose.Pose()
            mp_drawing = mp.solutions.drawing_utils

            feature_names = [f'keypoint_{i}_{axis}' for i in range(33) for axis in ['x', 'y', 'z', 'visibility']]

            cap = cv2.VideoCapture(0)

            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(frame_rgb)

                if results.pose_landmarks:
                    keypoints = []
                    for landmark in results.pose_landmarks.landmark:
                        keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
                else:
                    keypoints = [0] * len(feature_names)

                keypoints_df = pd.DataFrame([keypoints], columns=feature_names)
                keypoints_scaled = scaler.transform(keypoints_df)
                pose_probabilities = model.predict_proba(keypoints_scaled)[0]
                threshold = 0.5

                font_scale_detected = 1.5
                font_thickness_detected = 2
                font = cv2.FONT_HERSHEY_SIMPLEX
                x, y = 10, 50

                detected_pose = np.argmax(pose_probabilities)
                detected_prob = pose_probabilities[detected_pose]

                if detected_prob >= threshold:
                    detected_text = f'Detected Pose: {class_names[detected_pose]} ({detected_prob:.2f})'
                else:
                    detected_text = 'No pose detected'

                (text_width, text_height), baseline = cv2.getTextSize(detected_text, font, font_scale_detected, font_thickness_detected)
                rect_x1, rect_y1 = x, y - text_height - baseline
                rect_x2, rect_y2 = x + text_width, y + baseline
                cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), cv2.FILLED)
                cv2.putText(frame, detected_text, (x, y), font, font_scale_detected, (255, 255, 255), font_thickness_detected, cv2.LINE_AA)

                # Prepare text for each class
                font_scale_table = 1
                font_thickness_table = 2
                table_start_y = y + text_height + 20  # Start position for the table
                line_height = 40  # Height between each line

                for i, class_name in enumerate(class_names):
                    prob = pose_probabilities[i]
                    text = f'{class_name}: {prob:.0%}'

                    # Draw background rectangle for text
                    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale_table, font_thickness_table)
                    rect_x1, rect_y1 = x, table_start_y + i * line_height - text_height - baseline
                    rect_x2, rect_y2 = x + text_width, table_start_y + i * line_height + baseline
                    cv2.rectangle(frame, (rect_x1, rect_y1), (rect_x2, rect_y2), (0, 0, 0), cv2.FILLED)

                    # Display text
                    cv2.putText(frame, text, (x, table_start_y + i * line_height), font, font_scale_table, (255, 255, 255), font_thickness_table, cv2.LINE_AA)

                # Draw the pose landmarks on the frame
                if results.pose_landmarks:
                    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # Display the frame
                cv2.imshow('Pose Recognition', frame)
    
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            cap.release()
            cv2.destroyAllWindows()
    
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

            
            
if __name__ == "__main__":
    root = tk.Tk()
    app = PoseApp(root)
    root.mainloop()

I0000 00:00:1718749625.417666       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
